# Preparation
Firstly, I will repeat the steps in the example notebook to get the data and do some basic data cleaning.

In [ ]:
import numpy as np
from pymatgen.io.cif import CifParser
from urllib.request import urlopen

request = urlopen("https://raw.githubusercontent.com/sheriftawfikabbas/crystalfeatures/master/Li10Ge(PS6)2_mp-696128_computed.cif")
cifFile = request.read().decode('utf-8')
parser = CifParser.from_string(cifFile)

structure = parser.get_structures()
# Returns a list of Structure objects
# #http://pymatgen.org/_modules/pymatgen/core/structure.html
# Let's print the first (and only) Structure object
print(structure[0])

from pymatgen.ext.matproj import MPRester
from pymatgen.ext.matproj import MPRestError

m = MPRester("Ua7LfrKkn9yTWA3t")
results=m.query({"band_gap": {"$gt": 6}},properties=["material_id"])